# Setup

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

True

## Load vector database

In [2]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Sample query

In [3]:
# query = 'What types of lubricants are to be avoided for mechanisms design?'
# query = 'What are examples of harmonic drive gearboxes for aerospace applications?'
# query = 'What types of deployable decelerators are there'
# query = 'What can you tell me about the Orion Side Hatch Design? Please explain any failures and lessons learned in detail'
# query = 'What can you tell me about ball-lock mechanism failures? Refer to specific examples.'

query = 'What can you tell me about latch mechanism design failures which have occurred'
query_followup = "Which programs or vehicles did these failures occur on, using the chat history as context"

In [4]:
docs = vectorstore.similarity_search(query,k=6)
# docs_score = vectorstore.similarity_search_with_relevance_scores(query,k=4)

In [5]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed 
the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and 
fully, in the direction of release. A cause for this behavior could not be found initially. Physical and 
dimensional inspection of the parts did not reveal any clear discrepancies. The engineering analysis 
77


## Load LLM

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)

# Define prompt templates

In [7]:

from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

## Define a standard prompt to use

In [8]:
template = """Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."
----------------
{context}
----------------
{chat_history}
Human:{human_input}
Chatbot:"""

full_template = (
    "Here are your instructions to answer that you MUST ALWAYS Follow: "
    + template
)
# messages = [
#     SystemMessagePromptTemplate.from_template(full_template),
#     HumanMessagePromptTemplate.from_template("{human_input}"),
# ]
# CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=full_template
)

## Initialize memory

In [9]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

# summary_memory = ConversationSummaryMemory(llm=OpenAI())
# memory = ConversationBufferMemory()

# memory = ConversationBufferMemory(memory_key="chat_history", 
#                                   input_key="human_input")
memory = ConversationSummaryMemory(llm=OpenAI(),
                                           memory_key="chat_history",
                                           input_key="human_input")

# conversation_summary = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=summary_memory,
#     prompt=prompt
# )

# conversation = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=memory,
    
# )

## Initiate the chat and get a response

In [10]:
# Chain
# chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True)
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt, memory=memory)

# Run
# TODO: fix this since it assumes it's the first chat for chat_history
resp=chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)
print(resp['output_text'])



I can tell you that latch mechanism design failures can occur due to insufficient controls on critical features, inadequate engineering controls, multiple prevailing conditions, and process variation and scrap. Additionally, design intent must be clearly communicated in engineering drawings and proper identification and inspection of critical features is essential.


# Chat history and follow-up
Check out ConversationalRetrievalChain in /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py

https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs 

In [11]:
chain.memory.buffer

'\nThe human asked the AI about latch mechanism design failures. The AI replied that such failures can occur due to insufficient controls on critical features, inadequate engineering controls, multiple prevailing conditions, and process variation and scrap. To prevent such failures, design intent must be clearly communicated in engineering drawings and proper identification and inspection of critical features is essential.'

# Test conversational retrieval
https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db

In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory)
result = qa({"question": query})

result['answer']

' The engineering analysis of a latch mechanism design failure revealed that the kinematics of the latch were designed to follow a vertical trajectory in the final phase of latching, but during functional testing, the trajectory showed a motion of the roller towards the edge of the latch tab hook. This happened due to flexibility of the secondary drive elements of the latch that created a bigger than expected deflection of the latch linkages involved. It was concluded that the design concept of the latching system was not safe against inadvertent release of the mechanism. Additionally, inadequate engineering controls on critical features can cause a failure, and proper identification and inspection of critical features is essential.'

## Passing in chat history

### First question

In [13]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever())

chat_history = []
result = qa({"question": query, "chat_history": chat_history})

result['answer']

' The engineering analysis of a latch mechanism design failure revealed that the kinematics of the latch were designed to follow a vertical trajectory in the final phase of latching, but during functional testing, the trajectory showed a motion of the roller towards the edge of the latch tab hook. This happened due to flexibility of the secondary drive elements of the latch that created a bigger than expected deflection of the latch linkages involved. It was concluded that the design concept of the latching system was not safe against inadvertent release of the mechanism. Additionally, inadequate engineering controls on critical features can cause a failure, and proper identification and inspection of critical features is essential.'

### Follow up

In [14]:
chat_history = [(query, result["answer"])]
result = qa({"question": query_followup, "chat_history": chat_history})

print(result['answer'])

 The latch mechanism design failures occurred on the Orion Side Hatch development program.


### Adding search distance

In [15]:
vectordbkwargs = {"search_distance": 0.5}
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)
chat_history = []
result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})

result['answer']

' The engineering analysis of a latch mechanism design failure revealed that the kinematics of the latch were designed to follow a vertical trajectory in the final phase of latching, but during functional testing, the trajectory showed a motion of the roller towards the edge of the latch tab hook. This happened due to flexibility of the secondary drive elements of the latch that created a bigger than expected deflection of the latch linkages involved. It was concluded that the design concept of the latching system was not safe against inadvertent release of the mechanism. Additionally, inadequate engineering controls on critical features can cause a failure, and proper identification and inspection of critical features is essential.'

In [16]:
chat_history = [(query, result["answer"])]
result = qa({"question": query_followup, "chat_history": chat_history})

result['answer']

' The latch mechanism design failures occurred on the Orion Side Hatch development program.'

### Add sources

In [17]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# llm = OpenAI(temperature=0)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [18]:
# vectordbkwargs = {"search_distance": 0.01}
# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)
chat_history = []
# result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
# result = chain({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
result = chain({"question": query, "chat_history": chat_history})

print(query+'\n')
print(result['answer'])

What can you tell me about latch mechanism design failures which have occurred

 Latch mechanism design failures have been observed in several cases, including insufficient controls on critical features, flexibility of the secondary drive elements of the latch, and the latch roller snapping off the latch tab.
SOURCES: ../data/AMS_2006.pdf, ../data/AMS_2008.pdf, ../data/AMS_2010.pdf


In [19]:
chat_history = [(query, result["answer"])]
# result = chain({"question": query_followup, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
result = chain({"question": query_followup, "chat_history": chat_history})

print(query_followup+'\n')
print(result['answer'])

Which programs or vehicles did these failures occur on, using the chat history as context

 The sources AMS_2000.pdf, AMS_2006.pdf, and AMS_2008.pdf reference latch mechanism design failures in a ground panel seal, a quick-release lock, and a retract latch seal, respectively.
SOURCES: AMS_2000.pdf, AMS_2006.pdf, AMS_2008.pdf


### Combining these

In [59]:
from langchain.chains.llm import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff")

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# memory = ConversationSummaryMemory(llm=llm,
#                                    memory_key="chat_history",
#                                    input_key="query")

# Construct a ConversationalRetrievalChain with a streaming llm for combine docs
# and a separate, non-streaming llm for question generation
# streaming_llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)


In [60]:
from langchain.prompts.prompt import PromptTemplate

system_message="Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."

_template_condense = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Chat History:
{chat_history}
User Question: {question}
Standalone Question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template_condense)

_template_qa = """Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Sources and Context from Reference Documents:
{context}
User Question:{question}
Chatbot:

"""

QA_PROMPT = PromptTemplate.from_template(_template_qa)

In [61]:
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

# TODO: get this to work with custom prompt and load_qa_with_sources
# doc_chain = load_qa_with_sources_chain(llm,chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(), 
    combine_docs_chain=doc_chain, 
    question_generator=question_generator,
    return_source_documents=True,
    verbose=True
)

In [64]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

print(query+'\n')
print(result['answer']+'\n\n'+'Sources:'+'\n')

# print(result['answer'])
for data in result['source_documents']:
    print(data.metadata)



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
What can you tell me about latch mechanism design failures which have occurred

I can tell you that latch mechanism design failures have occurred due to insufficient controls on critical features, inadequate engineering controls, and discrepancies between parts from different lots. Vibration anomalies have been observed, and root cause and all contributing causes must be identified before the investigation process is concluded.

Sources:

{'page': 91.0, 'source': '../data/AMS_2006.pdf'}
{'page': 145.0, 'source': '../data/AMS_2008.pdf'}
{'page': 370.0, 'source': '../data/AMS_2010.pdf'}
{'page': 94.0, 'source': '../data/AMS_2006.pdf'}


In [65]:
chat_history = [(query, result["answer"])]
query_followup='Provide details on the inadequate engineering controls on critical features'

result = qa({"question": query_followup, "chat_history": chat_history})

print(query_followup+'\n')
print(result['answer']+'\n\n'+'Sources:')

# print(result['answer'])
for data in result['source_documents']:
    print(data.metadata)



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
Provide details on the inadequate engineering controls on critical features

The inadequate engineering controls on critical features related to latch mechanism design failures include not having a thorough understanding of which features are critical to performance, not having adequate engineering controls in place, and not properly identifying and inspecting critical features. Additionally, not ensuring a proper corner radius is specifically called out at corners of thread reliefs, especially on shoulder bolts subject to cyclic loading, can lead to design failures.

Sources:
{'page': 94.0, 'source': '../data/AMS_2006.pdf'}
{'page': 145.0, 'source': '../data/AMS_2008.pdf'}
{'page': 91.0, 'source': '../data/AMS_2006.pdf'}
{'page': 99.0, 'source': '../data/AMS_2006.pdf'}


In [67]:
chat_history.append((query_followup,result["answer"]))
query_followup='Provide details discrepancies between parts in different lots'

result = qa({"question": query_followup, "chat_history": chat_history})

print(query_followup+'\n')
print(result['answer']+'\n\n'+'Sources:')

# print(result['answer'])
for data in result['source_documents']:
    print(data.metadata)



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
Provide details discrepancies between parts in different lots

The discrepancies between parts in different lots can be attributed to inadequate drawing controls, combined with variation in the manufacturing process (change of vendor, machine, or even machinist). This can lead to differences in performance between parts from different lots, even if they meet the same drawing requirements. Proper identification and inspection of critical features is essential, and in-process screening at the assembly level provides redundancy in detection of variation.

Sources:
{'page': 94.0, 'source': '../data/AMS_2006.pdf'}
{'page': 93.0, 'source': '../data/AMS_2006.pdf'}
{'page': 271.0, 'source': '../data/AMS_2014.pdf'}
{'page': 196.0, 'source': '../data/AMS_2010.pdf'}


In [68]:
result
# TODO: find a way to get the retrieval question here

{'question': 'Provide details discrepancies between parts in different lots',
 'chat_history': [('What can you tell me about latch mechanism design failures which have occurred',
   'The inadequate engineering controls on critical features related to latch mechanism design failures include not having a thorough understanding of which features are critical to performance, not having adequate engineering controls in place, and not properly identifying and inspecting critical features. Additionally, not ensuring a proper corner radius is specifically called out at corners of thread reliefs, especially on shoulder bolts subject to cyclic loading, can lead to design failures.'),
  ('Provide details discrepancies between parts in different lots',
   'The discrepancies between parts in different lots can be attributed to inadequate drawing controls, combined with variation in the manufacturing process (change of vendor, machine, or even machinist). This can lead to differences in performance 

# Claude 2

In [ ]:
# TODO: test out claude 2

# Stuff to try and things not quite working

In [ ]:
# Add something with a system description so the chatbot knows some context.
# Try using the multi-context question tool as an input to a chain query.
# Use cosine scores to find more docs which are relevant.

# Try to create an agent to do a more extensive search.
#   Follow up from original prompt with a new question
#   Search for new docs based on the question
#   Plug new docs into another prompt with that question

In [ ]:
# TODO: #1 Check this out: https://python.langchain.com/docs/modules/data_connection/retrievers/ 
# TODO: #2 Check out how quivr uses langchain, copy key functionality

### Fancy stuff that is kinda sorta working

In [ ]:
# Generate
# https://python.langchain.com/docs/use_cases/question_answering/

# Return source docs
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)
qa_chain = RetrievalQA.from_llm(llm,retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)

results = qa_chain({'query': query})

In [ ]:
print(results['query']+'\n')
print(results['result']+'\n')

print('Sources:')
for doc in results['source_documents']:
    print(doc.metadata)